## I. Setup
The majority of the code for generating the `WireCell` noise model inputs is placed in the `utilities.py` module. Additionally, we will need access to the `PyROOT` libraries as the noise spectra from the analysis module are stored in a ROOT file. There is some path manipulation that must be done to point the Python kernel at the proper `PyROOT` library area.

In [1]:
from utilities import get_blocks_tpc, write_blocks
import sys

sys.path.append('/Users/mueller/ROOT/install/lib/')

from ROOT import TFile, TH2F

Welcome to JupyROOT 6.26/04


## II. Background
`WireCell` uses the FFT spectra of both the coherent and intrinsic noise components as input to the noise model. Each group of 32 comprised of `wired` channels is configured to have its own distinct spectra for each of the coherent and intrinsic components. The analysis module that produces these spectra has already done the down-sampling of 32 channels to a single spectra, so all that is required is to retrieve each group and place it in the correct location and format within the `WireCell` configuration file. Code from Sergey Martynenko was adapted to work with this particular analysis output format.

Each TPC and each component has a separate JSON configuration file (total = 8), which must be compressed prior to handing it to `WireCell`. The helper methods from `utilities.py` handle the formatting. This JSON format is documented below:

```
[
    {
        "period": 400.0,
        "nsamples": 4096,
        "gain": 8.811970678500002e-10,
        "shaping": 1.3,
        "wire-delta": 32.0,
        "const": 0.0,
        "tpcname": "EE",
        "freqs": [
            0.0,
            6.106497313141183e-07,
            ... (4096 bins)
        ]
        "amps": [
            9.97172161172161e-06,
            7.306608207059867e-07,
            ... (4096 bins)
        ]
    }
    ... (416 groups of 32 per TPC)
]

```
The keys are as follows:
* `period` - The duration of a single time tick [ns].
* `nsamples` - The duration of a single waveform [ticks].
* `gain` - The gain for the group of wires (constant here).
* `shaping` - The shaping time [us].
* `wire-delta` - The number of channels spanned by this block (constant at 32 here).
* `const` - Additional offset in frequency space [bins] (constant at 0.0 here).
* `tpcname` - The name of the TPC (EE, EW, WE, WW).
* `freqs` - The frequency of each spectra bin. This is "folded" about the Nyquist frequency such that bin i is equal to bin 4096 - i.
* `amps` - The amplitude of each spectra bin. This is similarly "folded" about the Nyquist frequency.

We must first load the ROOT file that we wish to use as the base for the noise model and extract the intrinsic and coherent spectra components. These are each stored as 2D histograms. The `get_blocks_tpc` function handles the organization of spectra and other parameters into a list of dictionaries for the requested TPC and component type. We can then write these blocks to a file using the `write_blocks` function. We can optionally choose to compress this to a `.bz2` file which, while necessary when using as input to `WireCell`, might not be useful for debugging.

In [3]:
input_file = TFile('/Users/mueller/data/noise/run10265.root', 'read')

for component in ['coh', 'int']:
    histogram = input_file.Get(f'{component}_ffts')
    for tpc in [0, 1, 2, 3]:
        tpc_name = {0: 'EE', 1: 'EW', 2: 'WE', 3: 'WW'}[tpc]
        blocks = get_blocks_tpc(histogram, tpc)
        write_blocks(blocks, f'/Users/mueller/data/icarus_noise_model_{component}_TPC{tpc_name}.json', compress=True)